In [29]:
import cv2
from PIL import Image, ImageDraw
import numpy as np

import base64

from flask import Flask, request
from flask_restful import Api, Resource, reqparse

import warnings
warnings.filterwarnings("ignore")

import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

# Стандартный opencv и каскады Хаара

In [5]:
face_cascade = cv2.CascadeClassifier('c:\python\haarcascade_frontalface_default.xml')

In [7]:
def start_cam():
    cam=cv2.VideoCapture(0)
    return cam
def stop_cam(cam):
    cam.release()


In [23]:
def get_face(cam):
    _,img1 =cam.read()
    img= cv2.flip(img1, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)==0:
        ret_image=np.uint8(np.zeros((25,25,3)))
        l,x,y,w,h=(0,-1,-1,25,25)
    else:
        wmax=-1
        l=len(faces)
        x,y,w,h=(-1,-1,-1,-1)
        for (xt,yt,wt,ht) in faces:
            if wmax<wt:
                wmax=wt
                x,y,w,h=(xt,yt,wt,ht)
        b,g,r=cv2.split(img[y:y+h, x:x+w])
        ret_image = cv2.merge((r,g,b))
        
    return(Image.fromarray(ret_image),l,x,y,w,h)


In [33]:
application = Flask(__name__)
api = Api(application)
cam=start_cam()




class GetFace(Resource):
    def get(self):
        pic,l,x,y,w,h=get_face(cam)
        img=base64.b64encode(pic.convert('RGB').tobytes()).decode()
        return {'Image':img,'Length':int(l),'X':int(x),'Y':int(y),'Width':int(w),'Height':int(h)}
api.add_resource(GetFace, '/GetFace')



class Shutdown(Resource):
    def get(self):
        func = request.environ.get('werkzeug.server.shutdown')
        if func is None:
            print ('Not running with the Werkzeug Server')
        else:
            func()
            print ('Server stopped')
            stop_cam(cam)
api.add_resource(Shutdown, '/Shutdown')



if __name__ == '__main__':
    application.run(host='0.0.0.0',port=8150)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Server stopped
